In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dasha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!wget https://raw.githubusercontent.com/semensorokin/DLforNLP_course_material/master/Homework2/answers_subsample.csv

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [3]:
!ls -l

total 28052
-rw-r--r-- 1 root root 28717126 Dec  1 19:58 answers_subsample.csv
drwxr-xr-x 1 root root     4096 Nov 23 14:31 sample_data


In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('answers_subsample.csv')

In [2]:
data

NameError: name 'data' is not defined

In [8]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

--2022-12-01 19:58:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  47.8MB/s    in 27s     

2022-12-01 19:59:25 (45.7 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [9]:
!ls -l

total 4458144
-rw-r--r-- 1 root root   28717126 Dec  1 19:58 answers_subsample.csv
-rw-r--r-- 1 root root 4536408847 Jan 18  2019 cc.ru.300.vec
drwxr-xr-x 1 root root       4096 Nov 23 14:31 sample_data


In [10]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 8.2 MB 13.2 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=69cb6e93ccc61de834a0c935ecb9269e3968d68571a4a97ad54845d449b4bbe8
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [11]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm
import pymorphy2
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def process_text(text):
    words = wordpunct_tokenize(text.lower())
    return words

In [19]:
import numpy as np

In [20]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('cc.ru.300.vec')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)


while True:
    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()
    current_word = ' '.join(current_parts[:-embedding_dim])
    if current_word in word2freq:

        word2index[current_word] = len(word2index)
        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)
        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()

word2vec_file.close()

vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████| 2000000/2000000 [01:15<00:00, 26517.59it/s]


In [44]:
from torch.utils.data import Dataset, DataLoader

In [47]:
cat_mapper = {cat: n for n, cat in enumerate(data.category.unique())}
data.category = data.category.map(cat_mapper)

In [50]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=32, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):      
        words = wordpunct_tokenize(text.lower())
        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            
            words = self.process_text(text)
            
            indexed_words = self.indexing(words)
            
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):
        
        return [self.word2index[word] for word in tokenized_text if word in self.word2index]
    
    def padding(self, sequence):

        if len(sequence)< self.sequence_length:
          add_pad = self.sequence_length - len(sequence)
          return sequence+[self.pad_index]*add_pad
        else:
          return sequence[:self.sequence_length]
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [52]:
x_train, x_validation, y_train, y_validation = train_test_split(data.text, data.category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=64)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=64)

Loading data: 100%|██████████| 23778/23778 [00:00<00:00, 108605.32it/s]


In [81]:
from math import sqrt

class model_with_att(torch.nn.Module):
  def __init__(self, matrix_w, n, emb_size): #n - количетсво категорий
        
        super().__init__()
        self.emb_size = emb_size
        self.n = n
        self.vectors = []

        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w))

        self.LSTM = torch.nn.LSTM(self.emb_size, 256, batch_first=True, dropout = 0.1, num_layers=2, bidirectional=True)
        
        self.q_proj = torch.nn.Linear(512, 400)
        self.k_proj = torch.nn.Linear(512, 400)
        self.v_proj = torch.nn.Linear(512, 400)

        self.att_soft = torch.nn.Softmax(dim = 2)
 
        self.cnn_3gr = torch.nn.Conv1d(400, 128, 3)
        self.cnn_4gr = torch.nn.Conv1d(400, 128, 4)
        self.cnn_5gr = torch.nn.Conv1d(400, 128, 5)

        self.linear_1 = torch.nn.Linear(384, 256)
        self.relu = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(256, out_features=n) 

        
  def forward(self, x):
      x_emb = self.emb_layer(x)
      x, _ = self.LSTM(x_emb) 

      x_q = self.q_proj(x) 
      x_k = self.k_proj(x)
      x_v = self.v_proj(x)

      att_scores = torch.bmm(x_q, torch.transpose(x_k, 2, 1)) / sqrt(self.emb_size)
      att_dist = self.att_soft(att_scores) 
      attention_vectors = torch.bmm(att_dist, x_v)

      x_att = attention_vectors.transpose(2,1) 

      x_cnn3 = self.cnn_3gr(x_att)
      x_cnn4 = self.cnn_4gr(x_att)
      x_cnn5 = self.cnn_5gr(x_att)

      frst, _ =  x_cnn3.max(dim= -1,)
      sc, _ = x_cnn4.max(dim= -1,)
      thr, _ = x_cnn5.max(dim= -1,)
      
      x_cat = torch.cat((frst, sc, thr), dim=-1) 
      
      x = self.linear_1(x_cat)
      x = self.relu(x)    
      x = self.linear_2(x)
    
      return x

In [57]:
n_classes = data.category.unique().shape[0]
model = model_with_att(vectors, n_classes, 300)

In [84]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [86]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(params=model.parameters(), lr = 0.001)

model = model.to(device)
criterion = criterion.to(device)

In [87]:
epochs = 10
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = model(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)

        with torch.no_grad():

            pred = model(x)

            pred = pred.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    #Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 1: 100%|██████████| 214001/214001 [01:26<00:00, 2464.65it/s, train_loss=0.476]



Losses: train - 0.560, test - 0.483
F1 test - 0.823


Epoch 2: 100%|██████████| 214001/214001 [01:27<00:00, 2445.44it/s, train_loss=0.443]



Losses: train - 0.457, test - 0.458
F1 test - 0.834


Epoch 3: 100%|██████████| 214001/214001 [01:26<00:00, 2480.13it/s, train_loss=0.418]



Losses: train - 0.430, test - 0.442
F1 test - 0.839


Epoch 4: 100%|██████████| 214001/214001 [01:26<00:00, 2467.22it/s, train_loss=0.396]



Losses: train - 0.407, test - 0.440
F1 test - 0.841


Epoch 5: 100%|██████████| 214001/214001 [01:26<00:00, 2477.26it/s, train_loss=0.375]



Losses: train - 0.385, test - 0.441
F1 test - 0.841
Early stopping


In [ ]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)